## Data

In [1]:
import numpy as np
import tensorflow as tf

from hopfield import LatentDenseRecon, DiscreteTimeHopfieldLayer, sign
from mnist import load_mnist

tf.random.set_seed(42)

tf.keras.backend.clear_session()

print(tf.__version__)

2.3.0


In [2]:
# IMAGE_SIZE = (32, 32)
IMAGE_SIZE = (8, 8)  # XXX: test!
BINARIZE = True
# BINARIZE = False

In [3]:
(x_train, _), _ = load_mnist(image_size=IMAGE_SIZE, binarize=BINARIZE)

## Model

In [4]:
def create_model():
    model = tf.keras.Sequential([
        DiscreteTimeHopfieldLayer(
            LatentDenseRecon(
                activation=tf.tanh,
                latent_dim=1024,
                binarize=sign,
            ),
            max_steps=20,
            reg_factor=1),
    ])
    model.compile(optimizer='adam')
    return model

In [5]:
X = x_train[:100].numpy()

ds0 = tf.data.Dataset.from_tensor_slices(X)
ds = ds0.shuffle(10000).repeat(10000).batch(128)
model = create_model()
model.fit(ds)

7813/7813 [==============================] - 36s 5ms/step - loss: 8.9369e-06


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
discrete_time_hopfield_layer (None, 64)                65537     
Total params: 65,537
Trainable params: 65,536
Non-trainable params: 1
_________________________________________________________________


In [97]:
# noised_X = X + np.random.normal(size=X.shape) * 0.3
noised_X = np.where(np.random.random(size=X.shape) < 0.3, -X, X)
recon_X = model.predict(noised_X)

for layer in model.layers:
    try:
        print('Relax steps:', layer.final_step.numpy())
    except AttributeError:
        pass

orig_err = noised_X - X
err = recon_X - X
print(f'{np.quantile(np.abs(orig_err), 0.99)} => '
      f'{np.quantile(np.abs(err), 0.99)}')

Relax steps: 2
2.0 => 2.0


In [53]:
X, recon_X

(array([[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]], dtype=float32),
 array([[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ...,  1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ...,  1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]], dtype=float32))

## Conclusions



### Methods of Dimensional Extension

* repeating: ×
* padding by given value: ×
* padding by random value: ×
* input grouping: √